In [2]:
import os
from dotenv import load_dotenv
from gitdb.fun import chunk_size

from ch07.main import rag_chain

load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

if not openai_api_key:
    raise ValueError("OpenAI API key not found.")

os.environ["OPENAI_API_KEY"] = openai_api_key




USER_AGENT environment variable not set, consider setting it to identify your requests.


In [6]:
import bs4
from langchain import hub
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI , OpenAIEmbeddings

# Load Documents

loader = WebBaseLoader(
    web_path=("https://news.naver.com/section/101",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("sa_text", "sa_item_SECTION_HEADLINE")
        )
    ),
)

docs = loader.load()

In [9]:
docs


[Document(metadata={'source': 'https://news.naver.com/section/101'}, page_content='\n\n코스피, 5000선 문턱서 숨 고르기…코스닥은 4년 만에 1000선 돌파\n\n코스피가 장중 5000선을 돌파했다가 하락 전환하며 숨 고르기에 들어간 반면, 코스닥은 4년 만에 1000선을 넘어서는 급등세를 연출했다. 대형주 중심의 랠리가 주춤해진 가운데 정책 기대와 순환매 흐름이 맞물리며 중\n\n\n디지털타임스\n\n\n\n\n69\n개의 관련뉴스 더보기\n\n\n\n\n\n해외 상장 등 플랜B도 쉽지 않아…재무적 투자자 엑시트 골머리 [시그널]\n\n이 기사는 2026년 1월 26일 17:07 자본시장 나침반 \'시그널(Signal)\' 에 표출됐습니다. LS가 중복상장 논란에 증손자회사 에식스솔루션즈의 상장심사를 전격 철회하면서 기업공개(IPO)로 자금을 조달해야\n\n\n서울경제\n\n\n\n\n64\n개의 관련뉴스 더보기\n\n\n\n\n\n설 연휴 전날 파생상품 야간거래 쉰다…거래소 2월 13일 휴장\n\n한국거래소가 설 연휴 전날 파생상품 야간거래를 쉬기로 했다. 장기 연휴 직전 가격 변동에 따른 투자자 리스크를 줄이기 위한 조치다. 한국거래소는 다음달 13일 오후 6시에 개시해 14일 오전 6시까지 예정됐던 파생상\n\n\n매일경제\n\n\n\n\n14\n개의 관련뉴스 더보기\n\n\n\n\n\n한강벨트 오를 때 지방은 침체…상하위 20% 격차 14배 벌어져[집슐랭]\n\n지난해 서울 주요 지역의 아파트 값이 가파르게 상승한 반면 비수도권 지역은 침체가 이어지면서 전국 아파트값 상위 20% 평균과 하위 20% 평균의 격차가 14배 이상 수준으로 벌어진 것으로 나타났다. 26일 한국부동\n\n\n서울경제\n\n\n\n\n32\n개의 관련뉴스 더보기\n\n\n\n\n\n“세계서 유례 찾기 힘든 경제형벌”…경제8단체, 배임죄 전면 개편 ‘한목소리’\n\n“대체입법 마련 시 배임죄 구성요건 명확히 

In [15]:
# Split
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=300,
    chunk_overlap=50
)

splits = text_splitter.split_documents(docs)

In [16]:
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

# MMR 알고리즘 (Maximal Magginal Relevance)
retriever = vectorstore.as_retriever(
    search_type="mmr",
    search_kwargs={'k': 1, 'fetch_k': 4}  # 상위 1개의 문서 반환, 고려할 문서는 4개 설정
)

In [17]:
# Prompt
prompt = hub.pull("sungwoo/ragbasic")

# LLM
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

# Post-processing
def format_docs(docs) :
    formatted = "\n\n".join(doc.page_content for doc in docs)
    return formatted

# Chain
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# Question
rag_chain.invoke("주식 관련 정보를 알려줘")


'현재 주식시장은 과열된 상태로, 코스피 지수가 5000선을 넘고 코스닥이 1000을 돌파했습니다. 이로 인해 유명 증권사 직원을 사칭한 불법 리딩방 사기가 증가하고 있으니 주의가 필요합니다. 투자 시 신뢰할 수 있는 정보원을 통해 확인하는 것이 중요합니다.'

In [18]:
# # 더 높은 다양성을 가진 문서 검색
#
# retriever = vectorstore.as_retriever(
#     search_type="mmr",
#     search_kwargs={'k': 6, 'lambda_mult': 0.25} # 상위 6개 문서 출력, 다양성 높이기 위한 lambda 값을 0.25 로 설정
# )
#
# # 더 많은 문서 고려
#
# retriever = vectorstore.as_retriever(
#     search_type="mmr",
#     search_kwargs={'k': 5, 'fetch_k': 50} # 50개의 문서를 고려하여 검색을 수행, 5개 문서 ㅁ반환
# )
#
# # 유사도 기반 검색
#
# retriever = vectorstore.as_retriever(
#     search_type="similarity_score_threshold", # 유사도 기반 점수 검색
#     search_kwargs={'score_threshold': 0.8} # 유사도 점수가 0.8 이상인 것만 검색
# )
#
# # 가장 유사한 하나만 검색
#
# retriever = vectorstore.as_retriever(
#     search_kwargs={'k': 1}
# )

# search_kwargs 등 파라미터를 통해 검색옵션을 변경 가능
- top-k similarity: 간단, 빠름, 질문-정답이 1:1 이라면 유용
- MMR:첫번째 결과만 보는게 아니라 다양한 논거와 예시가 필요하다면 유용.
    - lambda_mult 값을 0(유사도만) ~ 0.5(다양성 중시) 로 조정 가능.
    - fetch_k 를 k 보다 크게 두면 더 넓은 후보에서 2차 선별 후 최종 k 개 반환 -> 품질 안정

- hybrid: 키워드 검색과 벡터 검색 섞어정확한 다어 매칭과 의미적 유사도를 동시에 충족. 법률 조항번호, 상품코드처럼 정확한 토큰이 포함된 검색어에서 성능 좋아짐
- Similarity-Score Threshold: 유사도가 0.8 이상인 문서처럼, 정확도 필터 사용.

In [ ]:
import bs4
from langchain import hub
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI , OpenAIEmbeddings

# Load Documents

loader = WebBaseLoader(
    web_path=("https://news.naver.com/section/101",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("sa_text", "sa_item_SECTION_HEADLINE")
        )
    ),
)

docs = loader.load()

# Split
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=300,
    chunk_overlap=50
)

splits = text_splitter.split_documents(docs)

vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

# MMR 알고리즘 (Maximal Magginal Relevance)
retriever = vectorstore.as_retriever()

In [20]:
from langchain_core.prompts import ChatPromptTemplate

# Multi Query: Different Perspectives
# template = """You are an AI language model assistant. Your task is to generate five
# different versions of the given user question to retrieve relevant documents from a vector
# database. By generating multiple perspectives on the user question, your goal is to help
# the user overcome some of the limitations of the distance-based similarity search.
# Provide these alternative questions separated by newlines. Original question: {question}"""
# prompt_perspectives = ChatPromptTemplate.from_template(template)

template = """
당신은 AI 언어 모델 조수입니다. 당신의 임무는 주어진 사용자 질문에 대해 벡터 데이터베이스에서 관련 문서를 검색할 수 있도록 다섯 가지 다른 버전을 생성하는 것입니다.
사용자 질문에 대한 여러 관점을 생성함으로써, 거리 기반 유사성 검색의 한계를 극복하는 데 도움을 주는 것이 목표입니다.
각 질문은 새 줄로 구분하여 제공하세요. 원본 질문: {question}
"""
prompt_perspectives = ChatPromptTemplate.from_template(template)


from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

generate_queries = (
    prompt_perspectives
    | ChatOpenAI(model_name="gpt-4o-mini",temperature=0)
    | StrOutputParser()
    | (lambda x: x.split("\n"))
)

generated_query = generate_queries.invoke("집값의 향방?")
generated_query

['집값의 향후 전망은 어떻게 될까요?',
 '',
 '현재 집값의 추세와 앞으로의 변화는 어떤 영향을 받을까요?',
 '',
 '부동산 시장에서 집값이 오를지 내릴지에 대한 예측은 무엇인가요?',
 '',
 '향후 몇 년간 집값의 변동성에 대해 어떤 의견이 있나요?',
 '',
 '집값 상승 또는 하락의 주요 요인은 무엇이며, 앞으로의 방향성은 어떻게 될까요?']

In [22]:
from langchain.load import dumps, loads
def get_unique_union(documents: list[list]):
    """ 고유한 문서들의 합집합을 생성하는 함수"""

    # 리스트의 리스트를 평탄화하고, 각 문서를 문자열로 직렬화
    flattened_docs = [dumps(doc) for sublist in documents for doc in sublist]

    # 중복된 문서 제거
    unique_docs = list(set(flattened_docs))

    # 고유한 문서를 원래의 문서 객체로 변환하여 반환
    return [loads(doc) for doc in unique_docs]




In [23]:
question = "집값의 향방?"

retrieval_chain = generate_queries | retriever.map() | get_unique_union

docs = retrieval_chain.invoke({"question": question})
docs

/var/folders/2j/vyd5f6nn00b37z36r3jmx9fh0000gn/T/ipykernel_87328/2491430346.py:12: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  return [loads(doc) for doc in unique_docs]


[Document(metadata={'source': 'https://news.naver.com/section/101'}, page_content='조선비즈\n\n\n\n\n23\n개의 관련뉴스 더보기\n\n\n\n\n\n美 강력한 겨울폭풍…정전·항공편취소·천연가스값 폭등'),
 Document(metadata={'source': 'https://news.naver.com/section/101'}, page_content='이데일리\n\n2시간전\n\n\n\n\n\n\n\n\n친환경이냐? AI 강국이냐?‥정부 결국 "대형 원전 더 짓는다"'),
 Document(metadata={'source': 'https://news.naver.com/section/101'}, page_content='지난해 서울 주요 지역의 아파트 값이 가파르게 상승한 반면 비수도권 지역은 침체가 이어지면서 전국 아파트값 상위 20% 평균과 하위 20% 평균의 격차가 14배 이상 수준으로 벌어진 것으로 나타났다. 26일 한국부동\n\n\n서울경제\n\n\n\n\n32\n개의 관련뉴스 더보기'),
 Document(metadata={'source': 'https://news.naver.com/section/101'}, page_content='보신 것처럼 대통령이 직접 부동산 세금을 계속 언급하면서, 시장은 앞으로 어떤 상황이 펼쳐질지 주시하고 있습니다. 급매물이 시장에 나올 거란 기대와, 나올 매물이 많지 않을 거란 예측이 혼재하고 있습니다. 이지은 기\n\n\nKBS\n\n1시간전'),
 Document(metadata={'source': 'https://news.naver.com/section/101'}, page_content="이데일리\n\n2시간전\n\n\n\n\n\n\n\n\n'억지 내용' SNS 일방통보‥유독 동맹국 우리나라만 판 엎었다"),
 Document(metadata={'source': 'https://news.naver.com/section/101'}, page_

In [24]:
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough

template = """ 다음 맥락을 바탕으로 질문에 답변하세요
{context}

질문: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

final_rag_chain = (
    {"context": retrieval_chain,
     "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)


final_rag_chain.invoke(question)

'현재 집값의 향방에 대한 전망은 혼재된 의견이 존재합니다. 서울 주요 지역의 아파트 값은 지난해 가파르게 상승했지만, 비수도권 지역은 여전히 침체 상태입니다. 이로 인해 전국 아파트값 상위 20%와 하위 20%의 격차가 14배 이상으로 벌어졌습니다. \n\n또한, 정부의 부동산 정책에 대한 우려도 커지고 있습니다. 이재명 대통령의 정책이 과거 문재인 정권의 집값 폭등을 초래했던 정책과 유사하다는 분석이 나오고 있으며, 다주택자에 대한 세금 중과와 보유세 인상이 현실화될 가능성도 제기되고 있습니다. \n\n시장에서는 급매물이 나올 것이라는 기대와 매물이 많지 않을 것이라는 예측이 혼재하고 있으며, 일부는 "버티면 이긴다"는 분위기를 유지하고 있습니다. 따라서 집값의 향방은 정부 정책과 시장의 반응에 따라 달라질 것으로 보입니다.'